# Building conversational applications with Strands Agents

In this notebook, you learn how to use Strands agents to integrate external capabilities into conversational applications.

This is a conversion of the original module9-converse_with_tools.ipynb that used the Converse API with tool specifications. Instead, we'll use Strands agents for a more streamlined approach.

## Environment setup

In this task, you set up your environment and install the required packages for Strands agents.

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun
from strands import Agent, tool
from strands.models.bedrock import BedrockModel
from strands_tools import calculator

model_id = "amazon.nova-lite-v1:0"

## Define Tools for Strands Agents

In this task, we define the tools that our Strands agent will use. Instead of manually handling tool specifications and execution like in the original notebook, Strands agents handle this automatically.

In [2]:
@tool
def my_calculator_tool(oper1: int, oper2: int, operator: str) -> str:
    """
    Perform basic arithmetic operations on two numbers.
    
    Args:
        oper1: First operand
        oper2: Second operand  
        operator: Operation to perform (+, -, *, / or add, subtract, multiply, divide)
    
    Returns:
        Result of the calculation as a string
    """
    def add(a, b):
        return a + b

    def subtract(a, b):
        return a - b

    def multiply(a, b):
        return a * b

    def divide(a, b):
        if b == 0:
            return "Cannot divide by zero"
        else:
            return float(a / b)
        
    if operator == '+' or operator == 'add':
        return str(add(oper1, oper2))
    elif operator == '-' or operator == 'subtract':
        return str(subtract(oper1, oper2))
    elif operator == '*' or operator == 'multiply':
        return str(multiply(float(oper1), float(oper2)))
    elif operator == '/' or operator == 'divide':
        return str(divide(oper1, oper2))
    else:
        return "Invalid operator"

In [3]:
@tool
def duckduckgo_search(query: str) -> str:
    """
    Search with DuckDuckGo for information about a topic.
    
    Args:
        query: The search query
    
    Returns:
        DuckDuckGo search results as a string
    """
    tool = DuckDuckGoSearchRun()
    result = tool.invoke(query)
    return result

## Create Strands Agent

Now we create a Strands agent that uses the Bedrock LLM and our defined tools. The agent will automatically handle tool calling and conversation flow.

In [4]:
# Create the Bedrock LLM wrapper for Strands
llm = BedrockModel(
    model_id=model_id
)

# Create the agent with tools
agent = Agent(
    model=llm,
    tools=[my_calculator_tool, duckduckgo_search],
    system_prompt="You are a helpful assistant that can perform calculations and search for information on DuckDuckGo. Use the available tools when needed to provide accurate and helpful responses."
)

## Test the Strands Agent

Now let's test our Strands agent with the same query used in the original notebook.

In [5]:
# Test the agent with a complex query that requires both tools
query = "What is Amazon SageMaker? What is launch year multiplied by 2"

print(f"Query: {query}\n")
print("Agent Response:")
print("=" * 50)

# Run the agent
response = agent(query)
print(response)

Query: What is Amazon SageMaker? What is launch year multiplied by 2

Agent Response:
<thinking>First, I need to search for information about Amazon SageMaker to understand what it is. Then, I'll calculate the launch year multiplied by 2. The calculation doesn't require any external tools, but I'll use the calculator tool for consistency.</thinking>

Tool #1: duckduckgo_search
<thinking>I have the information about Amazon SageMaker. Now, I need to determine the launch year of Amazon SageMaker to multiply it by 2. I will search for the launch year of Amazon SageMaker.</thinking> 
Tool #2: duckduckgo_search

Tool #3: my_calculator_tool
<thinking>I have the information that Amazon SageMaker was launched in 2017. I can now calculate the launch year multiplied by 2. I already performed this calculation using the calculator tool, and the result is 4034.</thinking>


Tool #4: duckduckgo_search


Amazon SageMaker is a fully managed service offered by Amazon Web Services (AWS) that simplifies t

In [ ]:
# Test with a calculation-only query
calc_query = "What is 156 multiplied by 23?"
print(f"Query: {calc_query}")
print(f"Response: {agent(calc_query)}")
print("\n" + "="*50 + "\n")

# Test with a Wikipedia-only query
wiki_query = "Tell me about artificial intelligence"
print(f"Query: {wiki_query}")
print(f"Response: {agent(wiki_query)}")
print("\n" + "="*50 + "\n")

# Test with a complex query requiring both tools
complex_query = "What is an Astronomical unit (AU)? Express the circumference of the earth in AU"
print(f"Query: {complex_query}")
print(f"Response: {agent(complex_query)}")